<a href="https://colab.research.google.com/github/mateofuertes/ImageToGcode/blob/OCR-Method-2/training_ai_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install transformers torch datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load your data
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [ ]:
import pandas as pd
from datasets import Dataset

# Load data from CSV
csv_file_path = '/content/generated_data.csv'

# Read CSV file
df = pd.read_csv(csv_file_path, header=None, names=['text', 'label'])

# Display the DataFrame
print(df)

# Ensure there's enough data
print("DataFrame shape:", df.shape)
if df.shape[0] < 1:
    raise ValueError("Not enough data to create a training set.")

# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(df)

# Print the first few rows of the Dataset
print("Train dataset:", train_dataset[0])


FileNotFoundError: [Errno 2] No such file or directory: '/content/generated_data.csv'

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import DistilBertTokenizerFast

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Load data from CSV
csv_file_path = '/content/generated_data.csv (1)'
df = pd.read_csv(csv_file_path, header=None, names=['text', 'label'])

# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(df)

# Collect all labels from the training dataset
training_labels = [example['label'] for example in train_dataset]

# Print collected labels for debugging
print("Collected labels from train_dataset:", training_labels)

# Fit the encoder with the training labels
label_encoder.fit(training_labels)

# Print label classes to verify the fitting
print("Classes found by LabelEncoder:", label_encoder.classes_)
print(len(label_encoder.classes_))

# Define label2id and id2label mappings
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}
print("label2id mapping:", label2id)
print("id2label mapping:", id2label)

# Function to encode labels
def encode_labels(example):
    labels = example['label']
    try:
        # Transform the labels to encoded values
        encoded_labels = label_encoder.transform([labels])
        return {'label': encoded_labels[0]}  # Return the first and only element
    except ValueError as e:
        print(f"Error during label transformation: {e}")
        return {'label': -1}  # Return -1 for invalid labels

# Apply encoding to train dataset
train_dataset = train_dataset.map(encode_labels)

# Print the resulting train dataset
print(train_dataset.shape)


Collected labels from train_dataset: ['name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 'name', 'phone', 'email', 

Map:   0%|          | 0/3496 [00:00<?, ? examples/s]

(3496, 2)


In [ ]:
from transformers import DistilBertTokenizerFast
from torch.utils.data import DataLoader

# Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize the training dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create data loader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Print information about the DataLoader
print(f"Number of batches in train_loader: {len(train_loader)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/3496 [00:00<?, ? examples/s]

Number of batches in train_loader: 219


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, TrainingArguments, Trainer
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm

# Load and train the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
#     eval_dataset=test_dataset,
)

# Train the model
trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.403600


TrainOutput(global_step=657, training_loss=0.31114254440528255, metrics={'train_runtime': 476.618, 'train_samples_per_second': 22.005, 'train_steps_per_second': 1.378, 'total_flos': 1389466736197632.0, 'train_loss': 0.31114254440528255, 'epoch': 3.0})

In [ ]:
# Save the model
model.save_pretrained("business_card_model_3")
tokenizer.save_pretrained("business_card_tokenizer_3")

('business_card_tokenizer_3/tokenizer_config.json',
 'business_card_tokenizer_3/special_tokens_map.json',
 'business_card_tokenizer_3/vocab.txt',
 'business_card_tokenizer_3/added_tokens.json',
 'business_card_tokenizer_3/tokenizer.json')

In [ ]:
from google.colab import files
import shutil

# Create a zip file for easy downloading
shutil.make_archive('business_card_model_3', 'zip', 'business_card_model_3')
shutil.make_archive('business_card_tokenizer_3', 'zip', 'business_card_tokenizer_3')

# Download the zip files
files.download('business_card_model_3.zip')
files.download('business_card_tokenizer_3.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Testing testing testing
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load the model and tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained('/content/business_card_model_3')

# Function to predict the label for a given text
def predict_label(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Convert class index to label
    predicted_label = id2label[predicted_class]
    return predicted_label

# Test the model with example text
example_texts = ["Pe | eo University of Applied Sciences\n_ SY HOCHSCHULE Constantiaplatz 4 o 26723 Emden s Germany\n~ =< EMDEN:LEER  wwwhs-emden-leer.de\n\nM.Eng.\n\nJeffrey Wermann\n\nFaculty of Technology\n\nscientific assistant\n\ntel.: + 49 4921 807-19 48\n\nfax: + 49 4921 807-18 43\n\ne-mail: jeffrey wermann@hs-emden-leer.de\n\nTE\n"]

for text in example_texts:
    label = predict_label(text)
    print(f"Text: {text} -> Predicted Label: {label}")

Text: Pe | eo University of Applied Sciences
_ SY HOCHSCHULE Constantiaplatz 4 o 26723 Emden s Germany
~ =< EMDEN:LEER  wwwhs-emden-leer.de

M.Eng.

Jeffrey Wermann

Faculty of Technology

scientific assistant

tel.: + 49 4921 807-19 48

fax: + 49 4921 807-18 43

e-mail: jeffrey wermann@hs-emden-leer.de

TE
 -> Predicted Label: email
